The aim is toread 30 year daily rainfall data
1) plot PDF of rainfall for each longitude : at least for average of 10degrees ... so 36 PDFs for the globe (do this also of daily temperature)
2) compute ITCZ movement : location of centroid and its displacement sectorally
3) compute width of ITCZ
=> expectation: more signal over Land


It is copied from WK spectra code. ... just to see how data is loaded and read. Reset can be deleted


Atwood et al 2020: ITCZ centroid - the latitude at which the mean annual area-weighted tropical rainfall to the north equals that to the south, within the bounds 20°N to 20°S

I think this is same as what aiko voight's github code gives: check

In [ ]:
import numpy as np
import xarray as xr
# our local module:
import itcz

import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import xarray as xr
from pathlib import Path
import myfunctions as mf

In [ ]:
# =========================
# User-defined metadata
# =========================

varname = "pr"

# =========================
# Base CEDA paths
# =========================

CEDA_BASE = Path("/badc/cmip6/data/CMIP6")

In [ ]:
#Model Names
MODELS = {
    "UKESM1-0-LL":  {"institution": "MOHC",         "ensemble": "r1i1p1f2",  "grid": "gn",},
    # "CNRM-ESM2-1":  {"institution": "CNRM-CERFACS", "ensemble": "r1i1p1f2",  "grid": "gr",},
    # "MPI-ESM1-2-LR":{"institution": "MPI-M",        "ensemble": "r1i1p1f1",  "grid": "gn",},
    # "CESM2-WACCM":  {"institution": "NCAR",         "ensemble": "r1i1p1f1",  "grid": "gn",},
    # "IPSL-CM6A-LR": {"institution": "IPSL",         "ensemble": "r1i1p1f1",  "grid": "gr",},
}


In [ ]:
#Experiment details
EXPERIMENTS = {
    "HIST":     {"project": "CMIP",        "scenario": "historical", "color": "black"},
    "SSP245":   {"project": "ScenarioMIP", "scenario": "ssp245"},
    "SSP585":   {"project": "ScenarioMIP", "scenario": "ssp585"},
    "G6solar":  {"project": "GeoMIP",      "scenario": "G6solar"},
    "G6sulfur": {"project": "GeoMIP",      "scenario": "G6sulfur"},
}

In [ ]:
for model_name, model_meta in MODELS.items():

    var = {}
    var_by_year = {}

    # --- LOAD DATA FOR A MODEL ---
    for exp, meta in EXPERIMENTS.items():
        if exp not in ["SSP245", "G6sulfur", "G6solar"]:
            continue

        # open dataset
        # --- special-case ensemble override ---
        if model_name == "CESM2-WACCM":
            if meta["scenario"] == "G6sulfur":
                ensemble = "r1i1p1f2"
            else:
                ensemble = "r1i1p1f1"
        else:
            ensemble = model_meta["ensemble"]


        base = (
            CEDA_BASE
            / meta["project"]
            / model_meta["institution"]
            / model_name
            / meta["scenario"]
            / ensemble
            / "Amon"
            / varname
            / model_meta["grid"]
            / "latest"
        )

        print(str(base))
        # ds = mf.open_files(str(base))
        if model_name == "CESM2-WACCM":
            if meta["scenario"] == "G6sulfur":
                ds = mf.open_files_CESM_G6sulfur(base)
            elif meta["scenario"] == "ssp585":
                ds = mf.open_files_CESM_ssp585(base)
            else:
                ds = mf.open_files(str(base))
        elif model_name == "IPSL-CM6A-LR":
            if meta["scenario"] == "ssp585":
                ds = mf.open_files_IPSL_ssp585(base)
            else:
                ds = mf.open_files(str(base))
        else:
            ds = mf.open_files(str(base))
        
        # ds = ...
        var[exp] = mf.read_var(ds, varname)

    # --- ANNUAL MEAN FOR THIS MODEL ---
    for exp, da in var.items():
        var_by_year[exp] = {
            "ANN": mf.seasonal_mean_by_year(da, 1, 12)
        }
        londim = mf.get_lon_dim(da)


    itcz_by_exp = {}

    for exp in ["SSP245", "G6sulfur", "G6solar"]:
        da_ann = var_by_year[exp]["ANN"]
    
        itcz_by_exp[exp] = itcz.itcz_adam_lonwise(
            da_ann,
            lat=da_ann.lat,
            londim=londim,
            year_slice=slice(2071, 2101),
            latboundary=20,
            dlat=0.1,
        )


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

for ax, exp in zip(axes, ["SSP245", "G6sulfur", "G6solar"]):

    mean = itcz_by_exp[exp].mean(dim="year").values
    std  = itcz_by_exp[exp].std(dim="year").values

    # x axis: longitude values
    x = itcz_by_exp[exp][londim].values

    # optional: wrap longitudes to 0–360 if needed
    x = np.mod(x, 360)

    ax.bar(
        x,
        mean,
        yerr=std,
        capsize=2,
        alpha=0.8,
        width=(x[1]-x[0]),  # width matches grid spacing
    )

    ax.set_ylabel("ITCZ latitude (deg)")
    ax.set_title(exp)
    ax.set_ylim(-10, 10)
    ax.axhline(0, color="k", linewidth=0.5, linestyle="--")

axes[-1].set_xlabel("Longitude (deg)")
plt.tight_layout()
plt.show()


In [ ]:
plt.plot(itcz_by_exp["G6sulfur"].mean(dim="year").values- itcz_by_exp["SSP245"].mean(dim="year").values)
plt.plot(itcz_by_exp["G6solar"].mean(dim="year").values- itcz_by_exp["SSP245"].mean(dim="year").values)
# plt.plot(itcz_by_exp["G6sulfur"].mean(dim="year").values- itcz_by_exp["G6solar"].mean(dim="year").values)



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature

fig, axes = plt.subplots(
    3, 1,
    figsize=(14, 10),
    sharex=True,
    subplot_kw=dict(projection=ccrs.PlateCarree())
)

# --- latitude and longitude from model/grid ---
lat = da["lat"].values
lon = da["lon"].values

# --- wrap longitudes 0-360 → -180 to 180 for Cartopy ---
lon_wrap = ((lon + 180) % 360) - 180

# sort longitudes and keep sort index
sort_idx = np.argsort(lon_wrap)
lon_wrap_sorted = lon_wrap[sort_idx]

# create dummy background (0 values) matching sorted lon grid
dummy = np.zeros((lat.size, lon.size))

for ax, exp in zip(axes, ["SSP245", "G6sulfur", "G6solar"]):

    # ITCZ mean and spread
    mean = itcz_by_exp[exp].mean(dim="year").values
    std  = itcz_by_exp[exp].std(dim="year").values

    # --- wrap ITCZ longitudes same as grid and sort ---
    itcz_lon_wrap = ((itcz_by_exp[exp][londim].values + 180) % 360) - 180
    itcz_mean_sorted = mean[np.argsort(itcz_lon_wrap)]
    itcz_lon_sorted = np.sort(itcz_lon_wrap)

    # --- dummy background mesh ---
    lon2d, lat2d = np.meshgrid(lon_wrap_sorted, lat)
    ax.pcolormesh(lon2d, lat2d, dummy, alpha=0, shading='auto', transform=ccrs.PlateCarree())

    # --- plot ITCZ bars along equator ---
    ax.bar(
        itcz_lon_sorted,
        itcz_mean_sorted,
        yerr=std,  # optional
        capsize=2,
        alpha=0.8,
        width=(lon_wrap_sorted[1]-lon_wrap_sorted[0]),
        color="C0",
    )

    # equator line
    ax.axhline(0, color="k", linewidth=0.5, linestyle="--")

    # land/ocean
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN, facecolor='aliceblue')
    ax.coastlines(linewidth=0.6)

    # set global extent
    ax.set_xlim(-180, 180)
    ax.set_ylim(-30, 30)
    ax.set_title(exp)

axes[-1].set_xlabel("Longitude (deg)")
plt.tight_layout()
plt.show()


COMPUTE ITCZ SHIFTS

In [ ]:
import xarray as xr
import numpy as np

# ITCZ shift arrays (year × lon)
itcz_shift = {
    "G6sulfur": itcz_by_exp["G6sulfur"] - itcz_by_exp["SSP245"],
    "G6solar":  itcz_by_exp["G6solar"]  - itcz_by_exp["SSP245"]
}

# sanity check
for exp, data in itcz_shift.items():
    print(exp, data.shape)  # should be (30, 192) or whatever your shape is


In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

fig, axes = plt.subplots(
    2, 1,
    figsize=(14, 10),
    sharex=True,
    subplot_kw=dict(projection=ccrs.PlateCarree())
)

lat = da["lat"].values
lon = da["lon"].values
lon_wrap = ((lon + 180) % 360) - 180
sort_idx = np.argsort(lon_wrap)
lon_wrap_sorted = lon_wrap[sort_idx]

dummy = np.zeros((lat.size, lon.size))

for ax, exp in zip(axes, ["G6sulfur", "G6solar"]):

    mean_shift = itcz_shift[exp].mean(dim="year").values
    std_shift  = itcz_shift[exp].std(dim="year").values

    # Wrap and sort ITCZ shift longitudes same as projection
    itcz_lon_wrap = ((itcz_by_exp[exp][londim].values + 180) % 360) - 180
    sort_idx_itcz = np.argsort(itcz_lon_wrap)
    itcz_mean_sorted = mean_shift[sort_idx_itcz]
    itcz_std_sorted  = std_shift[sort_idx_itcz]
    itcz_lon_sorted  = np.sort(itcz_lon_wrap)

    # Dummy background for projection
    lon2d, lat2d = np.meshgrid(lon_wrap_sorted, lat)
    ax.pcolormesh(lon2d, lat2d, dummy, alpha=0, shading='auto', transform=ccrs.PlateCarree())

    # Plot ITCZ shift bars along equator
    ax.bar(
        itcz_lon_sorted,
        itcz_mean_sorted*10,
        yerr=itcz_std_sorted,
        capsize=2,
        alpha=0.8,
        width=(lon_wrap_sorted[1]-lon_wrap_sorted[0]),
        color="C1",
    )

    # Equator line
    ax.axhline(0, color="k", linewidth=0.5, linestyle="--")

    # Land/ocean features
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN, facecolor='aliceblue')
    ax.coastlines(linewidth=0.6)

    ax.set_xlim(-180, 180)
    ax.set_ylim(-30, 30)
    ax.set_title(f"ITCZ shift: {exp} - SSP245 *10")
    ax.set_ylabel("Latitude shift (deg) * 10")

axes[-1].set_xlabel("Longitude (deg)")
plt.tight_layout()
plt.show()
